In [2]:
pip install pandas sqlalchemy pymysql

Note: you may need to restart the kernel to use updated packages.


In [ ]:
import pandas as pd
from sqlalchemy import create_engine, text
import numpy as np

db_user = 'root'
db_password = 'AlumnaAdalab' 
db_host = '127.0.0.1'
database_name = 'Talento_ABC_Corporation' 
NOMBRE_ESQUEMA = 'Talento_ABC_Corporation' 
NOMBRE_CSV = 'hr_data_clean.csv' 

CADENA_CONEXION = f'mysql+pymysql://{db_user}:{db_password}@{db_host}:3306/{database_name}'


def normalizar_y_exportar_a_sql(archivo_csv, esquema, cadena_conexion):
    
  
    try:
        engine = create_engine(cadena_conexion)
        print("🔌 Conexión exitosa a la base de datos.")
        
        # --- PASO CLAVE: ELIMINAR TABLAS HIJAS PARA EVITAR ERROR DE FK ---
        # Borramos las tablas dependientes (ratings, compensation, background) para que 'employees' pueda borrarse.
        tablas_a_dropear_manualmente = ['ratings', 'compensation', 'background']
        
        with engine.connect() as connection:
            for tabla in tablas_a_dropear_manualmente:
                try:
                    # Ejecutar DROP TABLE IF EXISTS
                    connection.execute(text(f"DROP TABLE IF EXISTS {tabla};"))
                    connection.commit()
                    print(f"   Tabla '{tabla}' eliminada para permitir el borrado en cascada.")
                except Exception as e:
                    print(f"   ⚠️ Advertencia: No se pudo eliminar la tabla {tabla}. {e}")
            
    except Exception as e:
        print(f"❌ Error al conectar o preparar la BD: {e}")
        return

    # 2. Leer el CSV y preparar el DataFrame principal
    try:
        df_principal = pd.read_csv(archivo_csv)
        df_principal.columns = df_principal.columns.str.lower().str.replace('[^0-9a-zA-Z]+', '_', regex=True)
        print(f"✅ CSV leído. Total de filas: {len(df_principal)}")
    except FileNotFoundError:
        print(f"❌ Error: Archivo CSV no encontrado en '{archivo_csv}'")
        return

    # ----------------------------------------------------------------------
    ## 🔧 3. AJUSTE DE TIPOS DE DATOS (Para asegurar la precisión SQL)
    # ----------------------------------------------------------------------
    print("\n⏳ Ajustando tipos de datos y precisión...")
    
    # --- A. Fechas ---
    df_principal['datebirth'] = pd.to_datetime(df_principal['datebirth'], errors='coerce')

    # --- B. Decimales (Redondeo a 2 decimales para coincidir con DECIMAL(X, 2)) ---
    decimal_cols = [
        'totalworkingyears', 'yearssincelastpromotion', 
        'dailyrate_', 'hourlyrate_', 'monthlyincome_', 'monthlyrate_', 
        'salary_', 'percentsalaryhike', 'sameasmonthlyincome_'
    ]
    for col in decimal_cols:
        if col in df_principal.columns:
            df_principal[col] = pd.to_numeric(df_principal[col], errors='coerce').round(2)
    
    # --- C. Enteros Clave ---
    integer_cols = ['employeenumber', 'numberchildren', 'joblevel', 'education', 'trainingtimeslastyear', 'stockoptionlevel']
    for col in integer_cols:
        if col in df_principal.columns:
            df_principal[col] = pd.to_numeric(df_principal[col], errors='coerce').fillna(0).astype(pd.Int64Dtype())

    print("✅ Ajustes de tipos completados.")
    
    # ----------------------------------------------------------------------
    ## 🧩 4. NORMALIZACIÓN: Crear Claves Foráneas y Tablas
    # ----------------------------------------------------------------------
    
    # --- A. Normalizar job_roles (Tabla de Búsqueda) ---
    # La columna 'roledepartamentnumber' no existe en el CSV, así que no se selecciona.
    df_job_roles = df_principal[[
        'jobrole', 'department', 'joblevel' 
    ]].drop_duplicates().reset_index(drop=True).copy()
    
    # Creación de la Clave Primaria (PK) artificial: job_role_id
    df_job_roles['job_role_id'] = (df_job_roles.index + 1).astype(int)
    
    # --- B. FUSIONAR IDs: Asignar la FK (job_role_id) al DataFrame Principal ---
    df_principal = pd.merge(
        df_principal,
        df_job_roles[['jobrole', 'job_role_id']],
        on='jobrole',
        how='left'
    )
    
    # --- C. Definir los DataFrames por Tabla (Seleccionar columnas exactas) ---
    
    # 1. Tabla EMPLOYEES (Tabla Padre Principal)
    columnas_employees = [
        'employeenumber', 'gender', 'maritalstatus', 'datebirth', 'numberchildren', 'distancefromhome', 
        'numcompaniesworked', 'totalworkingyears', 'yearsatcompany', 'yearsincurrentrole', 
        'yearssincelastpromotion', 'yearswithcurrmanager', 'attrition', 'overtime', 
        'standardhours', 'remotework', 'education', 'job_role_id'
    ]
    df_employees = df_principal[columnas_employees].copy()
    # Mapeo de nombres de columna:
    df_employees = df_employees.rename(columns={'education': 'education_id', 'numberchildren': 'children'})
    
    # 2. Tabla BACKGROUND
    columnas_background = [
        'employeenumber', 'education', 'educationfield', 'trainingtimeslastyear',
        'numcompaniesworked', 'totalworkingyears'
    ]
    df_background = df_principal[columnas_background].copy()
    
    # 3. Tabla COMPENSATION
    columnas_compensation = [
        'employeenumber', 'dailyrate_', 'hourlyrate_', 'monthlyincome_',
        'monthlyrate_', 'salary_', 'percentsalaryhike', 'stockoptionlevel',
        'sameasmonthlyincome_'
    ]
    df_compensation = df_principal[columnas_compensation].copy()
    # Renombrar para quitar el '_' y coincidir con el SQL
    df_compensation.columns = [col.replace('_', '') if col.endswith('_') else col for col in df_compensation.columns]


    # 4. Tabla RATINGS
    columnas_ratings = [
        'employeenumber', 'environmentsatisfaction', 'jobinvolvement',
        'jobsatisfaction', 'relationshipsatisfaction', 'performancerating',
        'worklifebalance'
    ]
    df_ratings = df_principal[columnas_ratings].copy()
    
    # ----------------------------------------------------------------------
    ## 📤 5. EXPORTACIÓN a SQL 
    # ----------------------------------------------------------------------
    
    # El orden de la lista es crucial para la CREACIÓN: Padre primero.
    tablas_a_exportar = [
        (df_job_roles, 'job_roles'), 
        (df_employees, 'employees'), 
        (df_background, 'background'), 
        (df_compensation, 'compensation'), 
        (df_ratings, 'ratings') 
    ]
    
    for df_a_exportar, nombre_tabla in tablas_a_exportar:
        print(f"\n⚙️ Exportando tabla: {nombre_tabla}...")
        
        df_a_exportar.to_sql(
            name=nombre_tabla,
            con=engine,
            if_exists='replace', # Forzar la recreación de la tabla
            index=False 
        )
        print(f"✅ Tabla '{nombre_tabla}' exportada con éxito.")


# --- EJECUCIÓN DEL SCRIPT ---
normalizar_y_exportar_a_sql(NOMBRE_CSV, NOMBRE_ESQUEMA, CADENA_CONEXION)

🔌 Conexión exitosa a la base de datos.
   Tabla 'ratings' eliminada para permitir el borrado en cascada.
   Tabla 'compensation' eliminada para permitir el borrado en cascada.
   Tabla 'background' eliminada para permitir el borrado en cascada.
✅ CSV leído. Total de filas: 1614

⏳ Ajustando tipos de datos y precisión...
✅ Ajustes de tipos completados.

⚙️ Exportando tabla: job_roles...
✅ Tabla 'job_roles' exportada con éxito.

⚙️ Exportando tabla: employees...
✅ Tabla 'employees' exportada con éxito.

⚙️ Exportando tabla: background...
✅ Tabla 'background' exportada con éxito.

⚙️ Exportando tabla: compensation...
✅ Tabla 'compensation' exportada con éxito.

⚙️ Exportando tabla: ratings...
✅ Tabla 'ratings' exportada con éxito.
